In [1]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
import versor as vsr
import motor_jacobian

Populating the interactive namespace from numpy and matplotlib


In [2]:
rotor = (vsr.Biv(1,0,0).unit() * pi/6).exp()
rotor0 = vsr.Biv(pi/2, 0.0, 0.0).exp()
print(rotor)

Rot: [ 0.87 -0.5 0 0 ]


In [3]:
vectors_a = [vsr.Vec(1,0,0), vsr.Vec(0,1,0), vsr.Vec(0,0,1)]
vectors_b = [a.spin(rotor) for a in vectors_a]

In [4]:
n_points=10
vectors_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)) for i in range(n_points)]
vectors_b = [vector.spin(rotor) for vector in vectors_a]

In [5]:
 def jacrot(a, rot):
    j = np.zeros((3,3))
    for i in range(3):
        biv = vsr.Biv(0.0, 0.0, 0.0)
        biv[i] = 1.0
        j[:,i] = 2.0 * np.array(biv.comm(a.spin(rot))) # Wrong sign?
    return j


In [6]:
# motor_jacobian.diff_rotor_cost(rotor, a, b)

In [7]:
J = jacrot(vectors_a[0], rotor)
J

array([[ 0.5343, -2.946 ,  0.    ],
       [-1.1358,  0.    , -2.946 ],
       [ 0.    , -1.1358, -0.5343]])

In [8]:
def make_skew(a):
    temp = a[:,2].copy()
    a[:,2] = a[:,0].copy()
    a[:,0] = temp
    a[:,1] *= -1.0
#     a
#     a = np.roll(a,1)
    return a

In [9]:
make_skew(J) * 0.5

array([[ 0.    ,  1.473 ,  0.2672],
       [-1.473 , -0.    , -0.5679],
       [-0.2672,  0.5679,  0.    ]])

In [10]:
temp = J[:,2]

In [11]:
J[:,2] = J[:,0]
J[:,0] = temp
J

array([[ 0.    ,  2.946 ,  0.    ],
       [-2.946 , -0.    , -2.946 ],
       [-0.5343,  1.1358, -0.5343]])

In [12]:
def resrot(a,b,rot):
    return np.array(b - a.spin(rot)).reshape(-1,1)

In [13]:
a = vectors_a[0]
b = vectors_b[0]
J = jacrot(a, rotor0)
print(J)

[[ 0.7165 -2.946   0.    ]
 [ 1.0306  0.     -2.946 ]
 [ 0.      1.0306 -0.7165]]


In [14]:
a.spin(rotor0) ^ b * 2

Biv: [ -0.6822 3.191 -0.2684 ]

In [15]:
motor_jacobian.outer_product(a,b,rotor)

MemoryError: std::bad_alloc

In [16]:
f = resrot(a,b,rotor0)
print(f)

[[ 1.0832]
 [-0.0911]
 [ 0.    ]]


In [17]:
np.dot(J.T,f)

array([[ 0.6822],
       [-3.1912],
       [ 0.2684]])

In [18]:
a.spin(rotor0) ^ b

Biv: [ -0.3411 1.596 -0.1342 ]

In [19]:
a ^ b

Biv: [ 0.3411 0.07747 0.9212 ]

In [20]:
np.dot()

TypeError: Required argument 'a' (pos 1) not found

In [21]:
np.dot(J.T, J)

array([[ 1.5756, -2.1108, -3.0362],
       [-2.1108,  9.741 , -0.7385],
       [-3.0362, -0.7385,  9.1921]])

In [22]:
B

NameError: name 'B' is not defined

In [23]:
def Hess2(a,b,rot):
    H = np.zeros((3,3))
    B = a.spin(rot) ^ b
    for i in range(3):
        ei = vsr.Biv(0.0, 0.0, 0.0)
        ei[i] = 1.0
        H[:,i] = ei.comm(B)
    return H
Hess2(a,b,rotor0)

array([[ 0.    ,  0.1342,  1.5956],
       [-0.1342,  0.    ,  0.3411],
       [-1.5956, -0.3411,  0.    ]])

In [24]:
a

Vec: [ 0.52 -0.36 -1.5 ]

In [5]:
 def Hess(a,b, rot):
    j = np.zeros((3,3))
    R = vsr.EGA(rot)
    a = vsr.EGA(a)
    b = vsr.EGA(b)
    for i in range(3):
        ei = vsr.Biv(0.0, 0.0, 0.0)
        ei[i] = 1.0
        ei = vsr.EGA(ei)
        j[:,i] = -2.0 * np.array(((ei * a * R.rev() * b - b * R * a * ei.rev())))[4:7]
    return j
rotor = (vsr.Biv(1,0,0).unit() * pi/6).exp()
a = vsr.Vec(1,0,0)
b = a.spin(rotor)
Hess(a,b,rotor)
# np.sum([Hess(a,b,rotor0) for a,b in zip(vectors_a, vectors_b)], axis=0)

array([[-3.4641, -0.    , -0.    ],
       [-0.    , -3.4641,  2.    ],
       [-0.    , -2.    , -3.4641]])

In [18]:
ei

EGA: [ 0 0 0 0 0 0 1 0 ]

In [19]:
rotor0

Rot: [ 6.1e-17 -1 0 0 ]

In [75]:
j = np.zeros((3,3))
R = vsr.EGA(vsr.Biv(0,pi/6,0).exp())
a = vsr.EGA(vsr.Vec(1,0,0))
b = a.spin(vsr.EGA(rotor))
for i in range(3):
    ei = vsr.Biv(0.0, 0.0, 0.0)
    ei[i] = 1.0
    ei = vsr.EGA(ei)
#     print(((R * a * ei.rev()) ^ b)  + ((ei * a * R.rev()) ^ b) )
    j[:,i] = -2.0 * np.array(((R * a * ei.rev() + ei * a * R.rev()) ^ b))[4:7]
j

array([[-1.7321, -1.7321,  1.    ],
       [-0.    , -1.7321, -0.    ],
       [-0.    , -3.    , -0.    ]])

In [80]:
a.spin(R) ^ b * 2

EGA: [ 0 0 0 0 0.87 -0.87 -1.5 0 ]

In [82]:
B = vsr.EGA(vsr.Biv(1,0,0))

In [87]:
(b * R * B * a * R.rev() + b * R * a * B.rev() * R.rev()) * -0.5

EGA: [ 0.87 -0 -0 -0 0.5 -0 -0 -0 ]

In [ ]:
(b * R * B * a * R.rev() + b * R * a * B.rev() * R.rev()) * -0.5

In [76]:
(R.rev() * R) *( (a * R.rev() * b) - (R.rev() * (b * R * a) * R.rev()))

EGA: [ 0.43 0 0 0 1.5 -0.75 -0.87 0 ]

In [63]:
X = vsr.EGA(vsr.Biv(1,0,0)) * R
X

EGA: [ 0 0 0 0 0.87 0 0.5 0 ]

In [65]:
R

EGA: [ 0.87 0 0 0 0 -0.5 0 0 ]

In [71]:
X = vsr.EGA(vsr.Biv(1,0,0)) * R
b * X * a * R.rev() + b * R * a * X

EGA: [ 0 0 0 0 0 -1.5 0.87 0 ]

In [72]:
X = vsr.EGA(vsr.Biv(0,1,0)) * R
b * X * a * R.rev() + b * R * a * X

EGA: [ 0.43 0 0 0 -0.75 0.25 0.43 0 ]

In [73]:
X = vsr.EGA(vsr.Biv(0,0,1)) * R
b * X * a * R.rev() + b * R * a * X

EGA: [ 0 0 0 0 0 -0.87 0.5 0 ]

In [48]:
(R * a * R.rev() * b)

EGA: [ 0.25 0 0 0 0.43 -0.43 -0.75 0 ]

In [49]:
(R * ())

EGA: [ 0 0 0 0 0.87 -0.87 -1.5 0 ]

In [50]:
a.spin(R) ^ b * 2.0

EGA: [ 0 0 0 0 0.87 -0.87 -1.5 0 ]

In [52]:
a.spin(R) * b - b * a.spin(R)

EGA: [ 0 0 0 0 0.87 -0.87 -1.5 0 ]

In [27]:
# np.array([motor_jacobian.outer_product(a, b, rotor0)[:,1:] for a,b in zip(vectors_a, vectors_b)]).reshape(1,3)

In [28]:
# motor_jacobian.outer_product(a, b, rotor0)

In [29]:
a.spin(b)

EGA: [ 0 1.4 2.2 -3.2 0 0 0 -1.1e-16 ]

In [30]:
np.sum([motor_jacobian.outer_product(a, b, rotor0)[:,1:] for a,b in zip(vectors_a, vectors_b)], axis=0)

TypeError: Incompatible function arguments. The following argument types are supported:
    1. (vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)1, (short)2, (short)4> >, vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)1, (short)2, (short)4> >, vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)3, (short)5, (short)6> >) -> array_t<double>


In [31]:
np.array([motor_jacobian.outer_product(a, b, rotor0)[:,1:] for a,b in zip(vectors_a, vectors_b)])

TypeError: Incompatible function arguments. The following argument types are supported:
    1. (vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)1, (short)2, (short)4> >, vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)1, (short)2, (short)4> >, vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)3, (short)5, (short)6> >) -> array_t<double>


In [32]:
np.dot(J.T,J)

array([[ 1.5756, -2.1108, -3.0362],
       [-2.1108,  9.741 , -0.7385],
       [-3.0362, -0.7385,  9.1921]])

In [33]:
Hess(vectors_a[0], vectors_b[0], rotor0)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. versor_pybind11.EGA(arg0: float, arg1: float, arg2: float, arg3: float, arg4: float, arg5: float, arg6: float, arg7: float)
    2. versor_pybind11.EGA(arg0: versor_pybind11.EGA)
    3. versor_pybind11.EGA(arg0: versor_pybind11.Vec)
    4. versor_pybind11.EGA(arg0: versor_pybind11.Biv)
    5. versor_pybind11.EGA(arg0: versor_pybind11.Tri)
    6. versor_pybind11.EGA(arg0: versor_pybind11.Rot)

Invoked with: Rot: [ 6.1e-17 -1 0 0 ]

In [ ]:
a_ = vsr.EGA(a.spin(rotor0))
b_ = vsr.EGA(b)
ei = vsr.Biv(0.0, 0.0, 0.0)
ej = vsr.Biv(0.0, 0.0, 0.0)

In [ ]:
rotor0 * a *  vsr.Biv(1,0,0) 

In [ ]:
[np.array(-2 * a ^ b)[4:6] for 

In [ ]:
(a_ <= ei)

In [ ]:
(a_ <= ei) <= ei

In [ ]:
a_.comm(ei)

In [ ]:
vsr.Biv(1,0,0).rev() * vsr.Biv(1,0,0).rev()

In [ ]:
Hess(a,rotor0);

In [ ]:
J = np.array([jacrot(a, rotor0) for a in vectors_a]).reshape(-1,3)
g = np.array([resrot(a,b,rotor0) for a,b in zip(vectors_a, vectors_b)]).reshape(-1,1)
np.dot(J.T,g)

In [ ]:
[np.sum(np.array((a.spin(rotor0) ^ b)).reshape(-1,3), axis=0) for a,b in zip(vectors_a, vectors_b)]

In [ ]:
B1 = -2 * np.sum([np.array((a.spin(rotor0) ^ b)).reshape(3,1).T for a,b in zip(vectors_a, vectors_b)], axis=0).flatten()
B = vsr.Biv(*B1)
B

In [ ]:
np.array((rotor0.rev() * B).biv())

In [ ]:
rotor0.rev() * (vectors_a[1].spin(rotor0) ^ vectors_b[1])

In [ ]:
J = np.array([jac(a, rotor0) for a in vectors_a]).reshape(-1,3)
J

In [ ]:
np.dot(J.T,J)

In [ ]:
H = np.sum([Hess(a,b, rotor0) for a,b in zip(vectors_a, vectors_b)], axis=0)
H

In [ ]:
H = np.sum([np.array(a.spin(rotor0) ^ b) for a,b in zip(vectors_a, vectors_b)], axis=0)
H

In [ ]:
def Hess(a,b, rot):
    j = np.zeros((3,3))
    R = vsr.EGA(rot)
    a = vsr.EGA(a)
    b = vsr.EGA(b)
    for i in range(3):
        ei = vsr.Biv(0.0, 0.0, 0.0)
        ei[i] = 1.0
        ei = vsr.EGA(ei)
        j[:,i] = -2.0 * np.array(((ei * a * R.rev() * b + b * R * a * ei)))[4:7]
    return j
np.sum([Hess(a,b,rotor0) for a,b in zip(vectors_a, vectors_b)], axis=0)

In [ ]:
def Hess3(rot):
    rot = vsr.Rot(*rot)
    H = np.sum([Hess(a,b,rot) for a,b in zip(vectors_a, vectors_b)], axis=0)
    return H

In [ ]:
def Jac3(rot):
    rot = vsr.Rot(*rot)
    return np.array([jacrot(a, rot) for a in vectors_a]).reshape(-1,3)

In [ ]:
J = Jac3(rotor0)

In [ ]:
def Res3(rot):
    rot = vsr.Rot(*rot)
    return np.array([resrot(a,b,rot) for a,b in zip(vectors_a, vectors_b)]).reshape(-1,1)

In [ ]:
def update(vectors_a, vectors_b, rot):
#     H = -2 * np.sum([motor_jacobian.outer_product(a, b, rot)[:,1:] for a,b in zip(vectors_a, vectors_b)], axis=0)
    J = np.array([jacrot(a, rot) for a in vectors_a]).reshape(-1,3)
    g = np.array([resrot(a,b,rot) for a,b in zip(vectors_a, vectors_b)]).reshape(-1,1)
    JTg = np.dot(J.T,g)
    JTJ = np.dot(J.T,J)
#     print(JTJ)
    H = -JTJ
#     H = np.sum([Hess(a,b,rot) for a,b in zip(vectors_a, vectors_b)], axis=0)
    print(H)
#     print(rot)
#     print(JTg)
#     print(H-JTJ)
    B = np.dot(np.linalg.pinv(H),JTg)
#     print(B)
#     B = np.sum([np.array(a.spin(rot) ^ b) for a,b in zip(vectors_a, vectors_b)], axis=0) # steepest descent
    r = vsr.Biv(*B).exp() * rot
#     grad_norm = np.linalg.norm(JTg)
    err = np.dot(g.T,g)[0][0]
    return r, err

In [ ]:
r = (vsr.Biv(1,0.001, 0.001).unit() * pi/9).exp()
r0 = vsr.Rot(1,0,0,0)
r = r0
r, err = update(vectors_a, vectors_b, r)
rs = [r0, r]
n = 1

# for i in range(4):
while err > 1e-6:
    print(n)
    r, err = update(vectors_a, vectors_b, r)
    rs.append(r)
    n += 1

print(n)
print(r)

In [ ]:
from scipy.optimize import minimize

In [ ]:
minimize(Res3, np.array(rotor0))

# Motor

In [3]:
m0 = vsr.Mot(1,0,0,0,0,0,0,0)
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
print(motor)

Mot: [ 0.87 0 -0.5 0 -0.68 -0.43 -0.18 -0.25 ]


In [4]:
n_points=3
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() for i in range(n_points)]
points_b = [point.spin(motor) for point in points_a]

In [5]:
vsr.Pnt(1,2,3,0,0).spin(motor)

Pnt: [ -2.098 2 2.366 0 2.268 ]

In [6]:
no = vsr.Pnt(0,0,0,1,0)
D = np.zeros((3,3))
for i in range(3):
    B = vsr.Drv(0,0,0)
    B[i] = 1
    D[:,i] = np.array(B.comm(no))[:3]
print(D)

[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]


In [9]:
#J1 = motor_jacobian.diff_point(motor, points_a[0])
print(J1)

MemoryError: std::bad_alloc

In [10]:
# J2 = motor_jacobian.jacobian_exp(motor, vsr.Dll(0,0,0,0,0,0))
print(J2)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. versor_pybind11.Dll(arg0: float, arg1: float, arg2: float, arg3: float, arg4: float, arg5: float)
    2. versor_pybind11.Dll(arg0: versor_pybind11.Pnt, arg1: versor_pybind11.Pnt)
    3. versor_pybind11.Dll(arg0: versor_pybind11.Pnt, arg1: versor_pybind11.Vec)

Invoked with: 0, 0, 0, 0, 0, 0

In [11]:
np.dot(J1,J2)

NameError: name 'J1' is not defined

In [ ]:
B =  motor.log().biv()

In [1]:
a = points_a[0]
a

NameError: name 'points_a' is not defined

In [ ]:
R = motor.rot()

In [ ]:
vsr.Biv(1,0,0).comm(a.spin(R))

In [ ]:
vsr.Dll(1,0,0,0,0,0).comm(a.spin(motor))

In [ ]:
vsr.Vec(1,2,3).null().spin(motor)

In [ ]:
vsr.Vec(1,2,3).spin(motor.rot())

In [ ]:
def jac(a, mot):
    j = np.zeros((3,6))
    for i in range(6):
        dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
        dll[i] = 1.0
        j[:,i] = -2.0 * np.array(dll.comm(a.spin(mot)))[:3]
    return j

In [ ]:
a = points_a[0]
b = points_b[0]
jac(a,motor)

In [ ]:
def res(a,b,mot):
    return np.array(b - a.spin(mot))[:3].reshape(-1,1)

In [ ]:
J = jac(a, m0)
r = res(a,b,m0)
B = vsr.Dll(*np.dot(J.T,r))

In [ ]:
B

In [ ]:
[vsr.Dll(0,1,0,0,0,0).comm(B) 
 for B in [vsr.Dll(*np.dot(jac(a,m0).T,res(a,b,m0))) 
           for a,b in zip(points_a, points_b)]]

In [ ]:
a.spin(m) ^ b

In [ ]:
def update(points_a, points_b, mot):
    J = np.array([jac(a, mot) for a in points_a]).reshape(-1,6)
    g = np.array([res(a,b,mot) for a,b in zip(points_a, points_b)]).reshape(-1,1)
    JTg = np.dot(J.T,g)
    print(JTg)
    print()
    H = np.dot(J.T,J)
    print(H)
    print()
    B = -np.dot(np.linalg.pinv(H),JTg)
    B = -JTg
    r = vsr.Dll(*B).exp() * mot
    grad_norm = np.linalg.norm(JTg)
    err = np.dot(g.T,g)[0][0]
    return r, err, grad_norm

In [ ]:
m = m0
res(points_a[0], points_b[0], m)
m, err, gnorm = update(points_a, points_b, m)
errs = []
# for i in range(6):
while err > 1e-6:
    m, err, gnorm = update(points_a, points_b, m)
    errs.append(err)
print(m)

In [ ]:
plt.semilogy(errs)